In [ ]:

# 1. IMPORTS
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)


# 2. CARREGAR OS DADOS
job_postings = pd.read_csv("job_postings.csv")
job_skills   = pd.read_csv("job_skills.csv")

print("\n[INFO] Formato dos arquivos:")
print(f" job_postings: {job_postings.shape}")
print(f" job_skills  : {job_skills.shape}")


# 3. MERGE ENTRE VAGAS E SKILLS
df = pd.merge(job_postings, job_skills, on="job_link", how="inner")
df = df.dropna(subset=["job_skills", "job_level"])

print("\n[INFO] Após merge e remoção de nulos em job_skills/job_level:")
print(f" df: {df.shape}")


# 4. VARIÁVEL-ALVO BINÁRIA
mask = df["job_level"].isin(["Associate", "Mid senior"])
df_model = df[mask].copy()

map_level = {
    "Associate": 0,      # junior
    "Mid senior": 1      # senior
}
df_model["target"] = df_model["job_level"].map(map_level)

print("\n[INFO] Distribuição da variável alvo (0 = Junior, 1 = Senior):")
print(df_model["target"].value_counts().rename(index={0: "Junior (0)", 1: "Senior (1)"}))


# 5. PRÉ-PROCESSAMENTO DAS SKILLS (TF-IDF)
def preprocess_skills(text):
    skills = [s.strip() for s in str(text).split(",") if s.strip()]
    skills_clean = [s.replace(" ", "_") for s in skills]
    return " ".join(skills_clean)

df_model["skills_text"] = df_model["job_skills"].apply(preprocess_skills)

print("\n[EXEMPLO] skills_text da primeira vaga:")
print(df_model["skills_text"].iloc[0])

vectorizer = TfidfVectorizer(
    min_df=10,   # aparecer em pelo menos 10 vagas
    max_df=0.9   # ignora skills muito genéricas
)

X = vectorizer.fit_transform(df_model["skills_text"])
y = df_model["target"].values

print("\n[INFO] Shape da matriz TF-IDF (linhas = vagas, colunas = skills):")
print(f" X: {X.shape}")


# 6. TESTE
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("\n[INFO] Tamanho dos conjuntos:")
print(f" Treino: {X_train.shape[0]} vagas")
print(f" Teste : {X_test.shape[0]} vagas")


# 7. REGRESSÃO LOGÍSTICA BINÁRIA
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"  # ajuda no desbalanceamento
)
log_reg.fit(X_train, y_train)


# 8. AVALIAÇÃO
y_pred = log_reg.predict(X_test)

print("\n" + "="*70)
print(" RESULTADOS DO MODELO ")
print("="*70)

# acurácia geral
acc = accuracy_score(y_test, y_pred)
print(f"\n[Acurácia geral] {acc:.4f}")

# forma tabela
report_dict = classification_report(
    y_test, y_pred, output_dict=True, zero_division=0
)

report_df = pd.DataFrame(report_dict).T
report_df.index = report_df.index.map(
    lambda x: {"0": "Junior (0)", "1": "Senior (1)",
               "accuracy": "accuracy",
               "macro avg": "macro avg",
               "weighted avg": "weighted avg"}.get(x, x)
)

print("\n[Relatório de classificação]")
print(report_df[["precision", "recall", "f1-score", "support"]].round(3))

# matriz de confusão organizada
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm,
    index=["Real: Junior (0)", "Real: Senior (1)"],
    columns=["Pred: Junior (0)", "Pred: Senior (1)"]
)

print("\n[Matriz de confusão]")
print(cm_df)


# 9. SKILLS MAIS IMPORTANTES (INTERPRETAÇÃO)
feature_names = np.array(vectorizer.get_feature_names_out())
coef = log_reg.coef_[0]

# skills senior (coeficiente positivo)
idx_top_senior = np.argsort(coef)[::-1][:15]
top_senior = pd.DataFrame({
    "skill": feature_names[idx_top_senior],
    "coeficiente": coef[idx_top_senior]
})
top_senior["tipo"] = "Aumenta probabilidade de Senior (1)"

# skills junior (coeficiente negativo)
idx_top_junior = np.argsort(coef)[:15]
top_junior = pd.DataFrame({
    "skill": feature_names[idx_top_junior],
    "coeficiente": coef[idx_top_junior]
})
top_junior["tipo"] = "Aumenta probabilidade de Junior (0)"

print("\n" + "="*70)
print(" TOP SKILLS POR NÍVEL ")
print("="*70)

print("\n[Top 15 skills que aumentam probabilidade de SER SENIOR (1)]")
print(top_senior[["skill", "coeficiente"]].reset_index(drop=True).round(3))

print("\n[Top 15 skills que aumentam probabilidade de SER JUNIOR (0)]")
print(top_junior[["skill", "coeficiente"]].reset_index(drop=True).round(3))



[INFO] Formato dos arquivos:
 job_postings: (12217, 15)
 job_skills  : (12217, 2)

[INFO] Após merge e remoção de nulos em job_skills/job_level:
 df: (12212, 16)

[INFO] Distribuição da variável alvo (0 = Junior, 1 = Senior):
target
Senior (1)    10915
Junior (0)     1297
Name: count, dtype: int64

[EXEMPLO] skills_text da primeira vaga:
Machine_Learning Programming Python Scala Java Data_Engineering Distributed_Computing Statistical_Modeling Optimization Data_Pipelines Cloud_Computing DevOps Software_Development Data_Gathering Data_Preparation Data_Visualization Machine_Learning_Frameworks scikitlearn PyTorch Dask Spark TensorFlow Distributed_File_Systems Multi_node_Database_Paradigms Open_Source_ML_Software Responsible_AI Explainable_AI

[INFO] Shape da matriz TF-IDF (linhas = vagas, colunas = skills):
 X: (12212, 3392)

[INFO] Tamanho dos conjuntos:
 Treino: 9769 vagas
 Teste : 2443 vagas

 RESULTADOS DO MODELO 

[Acurácia geral] 0.7990

[Relatório de classificação]
              p